<img src="https://cdn.comet.ml/img/notebook_logo.png">

[Comet](https://www.comet.com/site/products/ml-experiment-tracking/??utm_source=accelerate&utm_medium=colab&utm_campaign=comet_examples) is an MLOps Platform that is designed to help Data Scientists and Teams build better models faster! Comet provides tooling to track, Explain, Manage, and Monitor your models in a single place! It works with Jupyter Notebooks and Scripts and most importantly it's 100% free to get started!

[Hugging Face Accelerate](https://github.com/huggingface/accelerate) was created for PyTorch users who like to write the training loop of PyTorch models but are reluctant to write and maintain the boilerplate code needed to use multi-GPUs/TPU/fp16.

Instrument Accelerate with Comet to start managing experiments, create dataset versions and track hyperparameters for faster and easier reproducibility and collaboration.

[Find more information about our integration with Accelerate](https://www.comet.com/docs/v2/integrations/ml-frameworks/transformers/?utm_source=accelerate&utm_medium=colab&utm_campaign=comet_examples)

Curious about how Comet can help you build better models, faster? Find out more about [Comet](https://www.comet.com/site/products/ml-experiment-tracking/?utm_source=accelerate&utm_medium=colab&utm_campaign=comet_examples) and our [other integrations](https://www.comet.com/docs/v2/integrations/overview/?utm_source=accelerate&utm_medium=colab&utm_campaign=comet_examples)

Get a preview for what's to come. Check out a completed experiment created from this notebook [here](https://www.comet.com/examples/comet-example-accelerate-notebook/0373dd068a484105b16c4053407f1bb2/?utm_source=accelerate&utm_medium=colab&utm_campaign=comet_examples).


# Install Dependencies

In [ ]:
%pip install -U "comet_ml>=3.44.0" torch torchvision tqdm "accelerate>=0.17.0"

# Login to Comet

In [ ]:
import comet_ml

comet_ml.login()

# Import Dependencies

In [ ]:
from accelerate import Accelerator
from torch.autograd import Variable
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define Parameters

In [ ]:
hyper_params = {"batch_size": 100, "num_epochs": 3, "learning_rate": 0.01}

# Load Data

In [ ]:
# MNIST Dataset
train_dataset = datasets.MNIST(
    root="./data/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = datasets.MNIST(
    root="./data/", train=False, transform=transforms.ToTensor()
)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=hyper_params["batch_size"], shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=hyper_params["batch_size"], shuffle=False
)

# Define Model and Optimizer

In [ ]:
accelerator = Accelerator(log_with="comet_ml")
accelerator.init_trackers(
    project_name="comet-example-accelerate-notebook", config=hyper_params
)
device = accelerator.device


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        return x


model = Net().to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params["learning_rate"])

model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

# Train a Model

In [ ]:
def train(model, optimizer, criterion, dataloader, epoch):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (images, labels) in enumerate(
        tqdm(dataloader, total=len(dataloader))
    ):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)
        pred = outputs.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability

        accelerator.backward(loss)
        optimizer.step()

        # Compute train accuracy
        batch_correct = pred.eq(labels.view_as(pred)).sum().item()
        batch_total = labels.size(0)

        total_loss += loss.item()
        correct += batch_correct

        # Log batch_accuracy to Comet; step is each batch
        accelerator.log({"batch_accuracy": batch_correct / batch_total})

    total_loss /= len(dataloader.dataset)
    correct /= len(dataloader.dataset)

    # Log data at an epoch level
    accelerator.get_tracker("comet_ml").tracker.log_metrics(
        {"accuracy": correct, "loss": total_loss}, epoch=epoch
    )

In [ ]:
# Train the Model
print("Running Model Training")

max_epochs = hyper_params["num_epochs"]
for epoch in range(max_epochs + 1):
    print("Epoch: {}/{}".format(epoch, max_epochs))
    train(model, optimizer, criterion, train_loader, epoch)

# End Experiment 

In [ ]:
accelerator.end_training()